# SetUp

In [1]:
#Import Libraries
import os
import pandas as pd
import statsmodels.api as sm
import numpy as np
from sklearn.utils import resample
import zepid
from zepid import load_sample_data
from zepid.causal.ipw import IPTW

In [3]:
def trial_sequence(estimand):
    """Initialize a trial sequence with specified estimand."""
    return {
        "estimand": estimand,
        "data": None,
        "ipw_model": None,
        "outcome_model": None
    }

def set_data(trial, data, id_col, period_col, treatment_col, outcome_col, eligible_col):
    """Prepare and store trial data with relevant columns."""
    trial["data"] = data[
        [id_col, period_col, treatment_col, "x1", "x2", "x3", "x4",
         "age", "age_s", outcome_col, "censored", eligible_col]
    ].copy()
    return trial

def summarize_trial(trial):
    """Print a structured summary of the trial object."""
    print(f"## Trial Sequence Object")
    print(f"## Estimand: {trial['estimand']}\n")

    print("## Data:")
    data = trial["data"]
    if data is not None:
        n_obs = len(data)
        n_patients = data["id"].nunique()
        print(f" - N: {n_obs} observations from {n_patients} patients")

        # Data preview with head and tail
        print("\nData Preview:")
        with pd.option_context('display.max_columns', None, 'display.expand_frame_repr', False):
            print(data.head(2).to_string())
            print("---")
            print(data.tail(2).to_string())

        # Variable types
        print("\nVariable Types:")
        print(data.dtypes.to_string())
    else:
        print(" - No data loaded")

    # IPW information
    print("\n## IPW for informative censoring:")
    print(" - No weight model specified" if trial["ipw_model"] is None
          else f" - Model: {trial['ipw_model']}")

    # Trial sequence instructions
    print("\n## Sequence of Trials Data:")
    print(" - Use set_expansion_options() and expand_trials() to construct the sequence of trials dataset.")

    # Outcome model information
    print("\n## Outcome model:")
    print(" - Not specified" if trial["outcome_model"] is None
          else f" - Model: {trial['outcome_model']}")

# Initialize target trials
trial_pp = trial_sequence(estimand="PP")
trial_itt = trial_sequence(estimand="ITT")

# Create output directories
for dir_path in [os.path.join(os.getcwd(), "trial_pp"),
                 os.path.join(os.getcwd(), "trial_itt")]:
    os.makedirs(dir_path, exist_ok=True)

# Load and prepare data
data_censored = pd.read_csv("data_censored.csv")

# Set up trials
trial_pp = set_data(trial_pp, data_censored, "id", "period", "treatment", "outcome", "eligible")
trial_itt = set_data(trial_itt, data_censored, "id", "period", "treatment", "outcome", "eligible")

# Display ITT trial summary
print("\n" + "#"*50)
summarize_trial(trial_itt)
print("#"*50 + "\n")


##################################################
## Trial Sequence Object
## Estimand: ITT

## Data:
 - N: 725 observations from 89 patients

Data Preview:
   id  period  treatment  x1        x2  x3        x4  age     age_s  outcome  censored  eligible
0   1       0          1   1  1.146148   0  0.734203   36  0.083333        0         0         1
1   1       1          1   1  0.002200   0  0.734203   37  0.166667        0         0         0
---
     id  period  treatment  x1        x2  x3        x4  age     age_s  outcome  censored  eligible
723  99       6          1   1 -0.033762   1  0.575268   71  3.000000        0         0         0
724  99       7          0   0 -1.340497   1  0.575268   72  3.083333        1         0         0

Variable Types:
id             int64
period         int64
treatment      int64
x1             int64
x2           float64
x3             int64
x4           float64
age            int64
age_s        float64
outcome        int64
censored       int64
e

# 3.1 Censoring due to treatment switching

In the R code, the ```set_switch_weight_model()``` function is used to handle censoring due to treatment switching. This involves specifying numerator and denominator models to calculate the probability of receiving treatment in the current period, with separate models for patients based on their previous treatment status.

To replicate this in Python, we can use logistic regression models from the ```statsmodels```library to estimate these probabilities. Here's how you can implement this:

In [ ]:
'''
Step 1: We define a helper function fit_logistic_regression to fit logistic regression models using statsmodels.
Step 2: The set_switch_weight_model function fits the numerator and denominator models based on the provided formulas and stores them in the trial object.
Step 3: We apply this function to the per-protocol trial (trial_pp) with the specified formulas.
'''


#Step 1
def fit_logistic_regression(formula, data):
    """Fit a logistic regression model."""
    model = sm.Logit.from_formula(formula, data)
    result = model.fit(disp=False)
    return result

#Step 2
def set_switch_weight_model(trial, numerator_formula, denominator_formula, save_path=None):
    """
    Set up the switch weight models for treatment switching.
    """
    data = trial["data"].copy()

    # Fit numerator model
    numerator_model = fit_logistic_regression(numerator_formula, data)

    # Fit denominator model
    denominator_model = fit_logistic_regression(denominator_formula, data)

    # Store models in the trial object
    trial["switch_weights"] = {
        "numerator_model": numerator_model,
        "denominator_model": denominator_model
    }

    # Optionally, save the models
    if save_path:
        # Create the directory if it doesn't exist
        os.makedirs(save_path, exist_ok=True)
        numerator_model.save(f"{save_path}/numerator_model.pickle")
        denominator_model.save(f"{save_path}/denominator_model.pickle")

    return trial

# Define formulas
numerator_formula = 'treatment ~ age'
denominator_formula = 'treatment ~ age + x1 + x3'

# Apply to the per-protocol trial
trial_pp = set_switch_weight_model(
    trial_pp,
    numerator_formula=numerator_formula,
    denominator_formula=denominator_formula,
    save_path='trial_pp/switch_models'
)

# Display switch weight models summary
print(trial_pp["switch_weights"]["numerator_model"].summary())
print(trial_pp["switch_weights"]["denominator_model"].summary())


                           Logit Regression Results                           
Dep. Variable:              treatment   No. Observations:                  725
Model:                          Logit   Df Residuals:                      723
Method:                           MLE   Df Model:                            1
Date:                Sun, 02 Mar 2025   Pseudo R-squ.:                 0.04144
Time:                        09:26:57   Log-Likelihood:                -480.24
converged:                       True   LL-Null:                       -501.01
Covariance Type:            nonrobust   LLR p-value:                 1.163e-10
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.8867      0.333      5.671      0.000       1.235       2.539
age           -0.0421      0.007     -6.213      0.000      -0.055      -0.029
                           Logit Regression Results 

# Step 3.2: Other Informative Censoring

For other types of informative censoring, the R code uses the ```set_censor_weight_model()``` function to estimate inverse probability of censoring weights (IPCW). In Python, we can similarly fit logistic regression models to estimate the probability of censoring.

In [ ]:
'''
Step 1: We add a not_censored column to indicate whether an observation is not censored.
Step 2: The set_censor_weight_model function fits numerator and denominator models to estimate the probability of not being censored, based on the provided formulas.
Step 3: We apply this function to both the per-protocol (trial_pp) and intention-to-treat (trial_itt) trials.
'''

def set_censor_weight_model(trial, censor_event_col, numerator_formula, denominator_formula, save_path=None):
    """
    Set up the censor weight models for informative censoring.
    """
    data = trial["data"].copy()

    # Create the censoring indicator (1 - censored)
    data['not_censored'] = 1 - data[censor_event_col]

    # Fit numerator model
    numerator_model = fit_logistic_regression(numerator_formula, data)

    # Fit denominator model
    denominator_model = fit_logistic_regression(denominator_formula, data)

    # Store models in the trial object
    trial["censor_weights"] = {
        "numerator_model": numerator_model,
        "denominator_model": denominator_model
    }

    # Optionally, save the models
    if save_path:
        # Ensure the directory exists before saving.
        os.makedirs(save_path, exist_ok=True)
        numerator_model.save(os.path.join(save_path, "numerator_model.pickle")) # Use os.path.join to create the full file path
        denominator_model.save(os.path.join(save_path, "denominator_model.pickle"))  # Use os.path.join to create the full file path

    return trial

# Define formulas
numerator_formula = 'not_censored ~ x2'
denominator_formula = 'not_censored ~ x2 + x1'

# Apply to the per-protocol trial
trial_pp = set_censor_weight_model(
    trial_pp,
    censor_event_col='censored',
    numerator_formula=numerator_formula,
    denominator_formula=denominator_formula,
    save_path='trial_pp/censor_models'
)

# Apply to the intention-to-treat trial
trial_itt = set_censor_weight_model(
    trial_itt,
    censor_event_col='censored',
    numerator_formula=numerator_formula,
    denominator_formula=denominator_formula,
    save_path='trial_itt/censor_models'
)

# Display censor weight models summary for ITT trial
print(trial_itt["censor_weights"]["numerator_model"].summary())
print(trial_itt["censor_weights"]["denominator_model"].summary())

                           Logit Regression Results                           
Dep. Variable:           not_censored   No. Observations:                  725
Model:                          Logit   Df Residuals:                      723
Method:                           MLE   Df Model:                            1
Date:                Sun, 02 Mar 2025   Pseudo R-squ.:                 0.02676
Time:                        09:26:57   Log-Likelihood:                -196.70
converged:                       True   LL-Null:                       -202.11
Covariance Type:            nonrobust   LLR p-value:                  0.001007
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.4481      0.141     17.415      0.000       2.173       2.724
x2            -0.4486      0.137     -3.278      0.001      -0.717      -0.180
                           Logit Regression Results 

# Step 4: Calculate Weights

In the R code, the ```calculate_weights()``` function is used to fit the models specified earlier and compute the inverse probability weights. To replicate this functionality in Python:

In [ ]:
'''
Step 1: Fit the Numerator and Denominator Models: Utilize the logistic regression models defined previously to estimate the probabilities of treatment and censoring.

Step 2: Compute the Weights: Calculate the stabilized weights based on the fitted models.

Step 3: Store and Summarize the Models: Save the model summaries for inspection and validation.
'''

def calculate_weights(trial, save_path=None):
    """
    Fit the weight models and calculate the inverse probability weights.
    """
    data = trial["data"].copy()

    # Ensure necessary columns are present
    required_cols = ['treatment', 'censored', 'age', 'x1', 'x2', 'x3']
    for col in required_cols:
        if col not in data.columns:
            raise ValueError(f"Column '{col}' is missing from the data.")

    # Fit treatment models
    # Numerator model: P(treatment | age)
    numerator_model = sm.Logit.from_formula('treatment ~ age', data).fit(disp=False)

    # Denominator model: P(treatment | age + x1 + x3)
    denominator_model = sm.Logit.from_formula('treatment ~ age + x1 + x3', data).fit(disp=False)

    # Predict probabilities
    data['p_treatment_num'] = numerator_model.predict(data)
    data['p_treatment_denom'] = denominator_model.predict(data)

    # Calculate treatment weights
    data['treatment_weight'] = data['p_treatment_num'] / data['p_treatment_denom']

    # Fit censoring models
    # Numerator model: P(not_censored | x2)
    data['not_censored'] = 1 - data['censored']
    censor_numerator_model = sm.Logit.from_formula('not_censored ~ x2', data).fit(disp=False)

    # Denominator model: P(not_censored | x2 + x1)
    censor_denominator_model = sm.Logit.from_formula('not_censored ~ x2 + x1', data).fit(disp=False)

    # Predict probabilities
    data['p_not_censored_num'] = censor_numerator_model.predict(data)
    data['p_not_censored_denom'] = censor_denominator_model.predict(data)

    # Calculate censoring weights
    data['censoring_weight'] = data['p_not_censored_num'] / data['p_not_censored_denom']

    # Combine weights
    data['ip_weight'] = data['treatment_weight'] * data['censoring_weight']

    # Store the updated data and models in the trial object
    trial["data"] = data
    trial["weight_models"] = {
        "treatment": {
            "numerator": numerator_model,
            "denominator": denominator_model
        },
        "censoring": {
            "numerator": censor_numerator_model,
            "denominator": censor_denominator_model
        }
    }

    # Optionally, save the models
    if save_path:
        # Create the directory if it doesn't exist
        os.makedirs(save_path, exist_ok=True)  # This line ensures the directory exists
        numerator_model.save(os.path.join(save_path, "treatment_numerator_model.pickle")) # Use os.path.join to create the full file path
        denominator_model.save(os.path.join(save_path, "treatment_denominator_model.pickle"))  # Use os.path.join to create the full file path
        censor_numerator_model.save(os.path.join(save_path, "censor_numerator_model.pickle")) # Use os.path.join to create the full file path
        censor_denominator_model.save(os.path.join(save_path, "censor_denominator_model.pickle"))  # Use os.path.join to create the full file path

    return trial


def show_weight_models(trial):
    """
    Display summaries of the weight models.
    """
    weight_models = trial.get("weight_models", {})

    if "treatment" in weight_models:
        print("\nTreatment Weight Models:")
        print("\nNumerator Model Summary:")
        print(weight_models["treatment"]["numerator"].summary())
        print("\nDenominator Model Summary:")
        print(weight_models["treatment"]["denominator"].summary())

    if "censoring" in weight_models:
        print("\nCensoring Weight Models:")
        print("\nNumerator Model Summary:")
        print(weight_models["censoring"]["numerator"].summary())
        print("\nDenominator Model Summary:")
        print(weight_models["censoring"]["denominator"].summary())

# Calculate weights for the per-protocol trial
trial_pp = calculate_weights(trial_pp, save_path='trial_pp/weight_models')

# Calculate weights for the intention-to-treat trial
trial_itt = calculate_weights(trial_itt, save_path='trial_itt/weight_models')

# Display weight model summaries for the ITT trial
show_weight_models(trial_itt)



Treatment Weight Models:

Numerator Model Summary:
                           Logit Regression Results                           
Dep. Variable:              treatment   No. Observations:                  725
Model:                          Logit   Df Residuals:                      723
Method:                           MLE   Df Model:                            1
Date:                Sun, 02 Mar 2025   Pseudo R-squ.:                 0.04144
Time:                        09:26:57   Log-Likelihood:                -480.24
converged:                       True   LL-Null:                       -501.01
Covariance Type:            nonrobust   LLR p-value:                 1.163e-10
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.8867      0.333      5.671      0.000       1.235       2.539
age           -0.0421      0.007     -6.213      0.000      -0.055      -0.029


# Step 5: Set Expansion Options

In this step, we define how the trial data should be expanded by specifying parameters such as the output method and chunk size. This setup prepares the trial object for the expansion process.

## ```set_expansion_options``` Function:
Adds an ```expansion_options``` dictionary to the trial object, containing the output function and chunk size.

## ```save_to_csv``` Function:
Generates a function that saves each data chunk to a CSV file in the specified directory. This function is assigned to the ```output_func``` parameter in ```set_expansion_options```

In [ ]:
def set_expansion_options(trial, output_func, chunk_size=500):
    """
    Set options for expanding the trial data.

    Parameters:
    - trial (dict): The trial object containing data and configurations.
    - output_func (function): Function to process and save each data chunk.
    - chunk_size (int): Number of patients to include in each expansion iteration.

    Returns:
    - dict: Updated trial object with expansion options set.
    """
    trial["expansion_options"] = {
        "output_func": output_func,
        "chunk_size": chunk_size
    }
    return trial

def save_to_csv(directory):
    """
    Creates a function to save DataFrame chunks to CSV files in the specified directory.

    Parameters:
    - directory (str): Directory path where CSV files will be saved.

    Returns:
    - function: A function that takes a DataFrame chunk and saves it as a CSV file.
    """
    def process_chunk(chunk, chunk_id):
        os.makedirs(directory, exist_ok=True)
        file_path = os.path.join(directory, f"expanded_data_chunk_{chunk_id}.csv")
        chunk.to_csv(file_path, index=False)
        print(f"Saved: {file_path}")
    return process_chunk

# Set expansion options for the per-protocol trial
trial_pp = set_expansion_options(
    trial_pp,
    output_func=save_to_csv("trial_pp/expanded_data"),
    chunk_size=500
)

# Set expansion options for the intention-to-treat trial
trial_itt = set_expansion_options(
    trial_itt,
    output_func=save_to_csv("trial_itt/expanded_data"),
    chunk_size=500
)


# Step 6: Expand Trials

With the expansion options set, this step involves processing the trial data in chunks as specified, applying the output function to each chunk to generate the expanded dataset.

##```expand_trials``` Function:
Retrieves the data and expansion options from the trial object, divides the data into chunks based on the specified chunk size, and applies the output function to each chunk. The ```output_func``` is called with the data chunk and its corresponding chunk index.

In [ ]:
def expand_trials(trial, censor_at_switch=True, first_period=0, last_period=10):
    """
    Expand the trial data to emulate a sequence of target trials.

    Parameters:
    - trial (dict): The trial object containing the original data.
    - censor_at_switch (bool): Whether to censor data at treatment switch.
    - first_period (int): The starting period for the trials.
    - last_period (int or float): The ending period for the trials.

    Returns:
    - dict: Updated trial object with expanded data.
    """
    data = trial.get("data")
    if data is None:
        raise ValueError("No data found in the trial object.")

    expanded_rows = []

    for _, patient_data in data.groupby('id'):
        patient_data = patient_data.sort_values(by='period')
        start_age = patient_data['age'].iloc[0]
        assigned_treatment = patient_data['treatment'].iloc[0]

        for trial_period in range(first_period, int(last_period) + 1):
            followup_time = 0
            for _, row in patient_data.iterrows():
                if censor_at_switch and row['treatment'] != assigned_treatment:
                    break

                expanded_row = row.copy()
                expanded_row['trial_period'] = trial_period
                expanded_row['followup_time'] = followup_time
                expanded_row['assigned_treatment'] = assigned_treatment
                expanded_rows.append(expanded_row)

                followup_time += 1

    expanded_data = pd.DataFrame(expanded_rows)
    trial['expanded_data'] = expanded_data

    return trial


# Step 6.1: Create Sequence of Trials Data


In [ ]:
# Example usage:
trial_pp = expand_trials(trial_pp)
trial_itt = expand_trials(trial_itt)

# Display the first few rows of the expanded data for the per-protocol trial
print(trial_pp['expanded_data'].head())

    id  period  treatment   x1        x2   x3        x4   age     age_s  \
0  1.0     0.0        1.0  1.0  1.146148  0.0  0.734203  36.0  0.083333   
1  1.0     1.0        1.0  1.0  0.002200  0.0  0.734203  37.0  0.166667   
2  1.0     2.0        1.0  0.0 -0.481762  0.0  0.734203  38.0  0.250000   
3  1.0     3.0        1.0  0.0  0.007872  0.0  0.734203  39.0  0.333333   
4  1.0     4.0        1.0  1.0  0.216054  0.0  0.734203  40.0  0.416667   

   outcome  ...  p_treatment_denom  treatment_weight  not_censored  \
0      0.0  ...           0.636513          0.930088           1.0   
1      0.0  ...           0.626528          0.928634           1.0   
2      0.0  ...           0.549849          1.039459           1.0   
3      0.0  ...           0.539206          1.040816           1.0   
4      0.0  ...           0.595948          0.924292           1.0   

   p_not_censored_num  p_not_censored_denom  censoring_weight  ip_weight  \
0            0.873678              0.914385         

# Step 7: Load or Sample from Expanded Data

After expanding the trial data, we prepare it for fitting the outcome model. If the dataset is manageable in size, we can load it directly into memory. For larger datasets, sampling may be necessary to ensure efficient processing.

In [ ]:
'''
In this snippet:

  We set a random seed for reproducibility.
  We define the proportion (p_control) of control observations (where outcome == 0) to include in the sampled data.
  We separate the treated and control observations.
  We sample from the control observations based on the defined proportion.
  We combine the treated observations with the sampled control observations to create the sampled_data DataFrame.
'''

# Assuming 'expanded_data' is your expanded DataFrame
# For large datasets, sample a subset; for small datasets, you might skip this step

# Set the random seed for reproducibility
random_seed = 1234

# Define the proportion of control observations to include
p_control = 0.5

# Access the 'expanded_data' from the trial dictionary
expanded_data = trial_pp['expanded_data']  # or trial_itt['expanded_data'], depending on which trial you want to sample from

# Separate treated and control observations
treated = expanded_data[expanded_data['outcome'] == 1]
control = expanded_data[expanded_data['outcome'] == 0]

# Sample from control observations
control_sampled = resample(control, replace=False, n_samples=int(p_control * len(control)), random_state=random_seed)

# Combine treated observations with sampled control observations
sampled_data = pd.concat([treated, control_sampled])

# Step 8: Fit Marginal Structural Model

To fit an MSM, we utilize inverse probability weighting (IPW). The ```zEpid``` library provides tools to calculate these weights and fit the MSM.

In [ ]:
'''
This example demonstrates how to fit a Marginal Structural Model (MSM) using zEpid's IPTW class:

1. Load the dataset. Replace load_sample_data(timevary=False) with your actual dataset.
2. Specify the exposure (treatment assignment) model using iptw.exposure_model().
3. Fit the exposure model using iptw.fit().
4. Specify the outcome model (MSM) using iptw.marginal_structural_model().
5. Fit the outcome model (MSM) using iptw.fit() again.
6. Access the results and summary.
'''

# Load your data
# For demonstration, we'll use zEpid's sample data
df = load_sample_data(timevary=False)

# Rename the 'dead' column to 'outcome' and 'art' to 'treatment'
df = df.rename(columns={'dead': 'outcome', 'art': 'treatment'})  # Assuming 'dead' is the outcome variable and 'art' is the treatment variable in the sample data

# Specify the exposure (treatment assignment) model
iptw = IPTW(df, treatment='treatment', outcome='outcome')
iptw.treatment_model('x2 + followup_time + I(followup_time**2) + trial_period + I(trial_period**2)')

# Specify the outcome model (MSM)
iptw.marginal_structural_model('assigned_treatment')
iptw.fit()

# Access results and summary
iptw.summary()



# Add a constant term for the intercept
df['intercept'] = 1.0

# Define the independent variables (including the intercept)
independent_vars = ['intercept', 'assigned_treatment', 'x2', 'followup_time', 'trial_period']

# Fit the weighted logistic regression model
model = sm.GLM(df['outcome'], df[independent_vars], family=sm.families.Binomial(), freq_weights=iptw.weights)
result = model.fit()

# Print the summary of the model
print(result.summary())


/usr/local/lib/python3.11/dist-packages/zepid/datasets/__init__.py:60: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(resource_filename('zepid', 'datasets/data.dat'),
/usr/local/lib/python3.11/dist-packages/zepid/datasets/__init__.py:60: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(resource_filename('zepid', 'datasets/data.dat'),
/usr/local/lib/python3.11/dist-packages/zepid/causal/utils.py:58: UserWarning: There is missing data that is not the outcome in the data set. IPTW will drop all missing data that is not missing outcome data. IPTW will fit 460 of 547 observations
  warnings.warn("There is missing data that is not the outcome in the data set. " + str(estimator) +


PatsyError: Error evaluating factor: NameError: name 'followup_time' is not defined
    treatment ~ x2 + followup_time + I(followup_time**2) + trial_period + I(trial_period**2)
                                     ^^^^^^^^^^^^^^^^^^^

In [ ]:
print(df.columns)


Index(['id', 'male', 'age0', 'cd40', 'dvl0', 'treatment', 'outcome', 't',
       'cd4_wk45'],
      dtype='object')
